In [1]:
# ==== Install Dependencies

!pip install -q efficientnet_pytorch
!pip install -q albumentations
!pip install -q pytorch-fanatics 
!pip install -q pytorch_ranger

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# ==== Import Libraries

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
import seaborn as sns
import random
import os


import albumentations as aug
from albumentations.pytorch.transforms import ToTensor
import matplotlib.pyplot as plt

from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset,DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pytorch_fanatics.dataloader import Cloader
from pytorch_fanatics.utils import EarlyStop 
from pytorch_fanatics.trainer import Trainer
from pytorch_fanatics.logger import Logger

import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from pytorch_ranger import Ranger

In [3]:
from glob import glob

In [4]:
train_path = []
train_label = []

val_path = []
val_label = []

In [5]:
for i in glob("../input/brain-tumor-classification-mri/Training/glioma_tumor/*"):
    train_path.append(i)
    train_label.append(0)
    
for i in glob("../input/brain-tumor-classification-mri/Training/meningioma_tumor/*"):
    train_path.append(i)
    train_label.append(1)

for i in glob("../input/brain-tumor-classification-mri/Training/no_tumor/*"):
    train_path.append(i)
    train_label.append(2)    
    
for i in glob("../input/brain-tumor-classification-mri/Training/pituitary_tumor/*"):
    train_path.append(i)
    train_label.append(3)

for i in glob("../input/brain-tumor-classification-mri/Testing/glioma_tumor/*"):
    val_path.append(i)
    val_label.append(0)
    
for i in glob("../input/brain-tumor-classification-mri/Testing/meningioma_tumor/*"):
    val_path.append(i)
    val_label.append(1)

for i in glob("../input/brain-tumor-classification-mri/Testing/no_tumor/*"):
    val_path.append(i)
    val_label.append(2)    
    
for i in glob("../input/brain-tumor-classification-mri/Testing/pituitary_tumor/*"):
    val_path.append(i)
    val_label.append(3)    
    

In [6]:
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [7]:
# ===== Augmentations

mean       = (0.485, 0.456, 0.406)
std        = (0.229, 0.224, 0.225)
train_tfms = aug.Compose([
            aug.Resize(224,224),
            aug.Flip(p=0.5),
            aug.ShiftScaleRotate(rotate_limit=(-45,45)),
            aug.GaussNoise(p=0.35),
            aug.Normalize(mean,std,max_pixel_value=255.0,always_apply=True),
            ])

test_tfms  = aug.Compose([
            aug.Resize(224,224),
            aug.Normalize(mean,std,max_pixel_value=255.0,always_apply=True),
            ])

In [8]:

train_dataset    = Cloader(train_path,train_label,None,train_tfms)
test_dataset     = Cloader(val_path,val_label,None,test_tfms)

train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True,num_workers=4)
val_dataloader   = DataLoader(test_dataset,batch_size=32,shuffle=False,num_workers=4)

device           = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
# ===== Define model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.base_model = EfficientNet.from_pretrained('efficientnet-b0',num_classes=4)
    def forward(self, image, targets):
        batch_size, _, _, _ = image.shape
        out = self.base_model(image)
        targets = torch.tensor(targets,dtype=torch.int64)
        loss = nn.CrossEntropyLoss()(out.view(batch_size,4), targets)
        return out, loss

model = Net()
model.to(device);

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


In [10]:
def softmax(array):
    return np.exp(array)/np.sum(np.exp(array),axis=1).reshape(-1,1)

In [11]:
optimizer = Ranger(model.parameters(),lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer,factor=0.8, mode="min",min_lr= 5e-6, patience=2)

trainer   = Trainer(model=model,optimizer=optimizer,device=device,val_scheduler=scheduler)
logger    = Logger()

es        = EarlyStop(patience=5,mode="min") # mode = min to minimise loss

In [12]:
epochs = 50
best_loss = np.Inf

for epoch in range(epochs):
    logger.write(f"+ ===== Epoch {epoch+1}/{epochs} ===== +")
    train_loss              = trainer.train(train_dataloader)
    y_true,y_pred ,val_loss = trainer.evaluate(val_dataloader)
    y_pred                  = softmax(y_pred)
    accuracy                = accuracy_score(y_true,np.argmax(y_pred,axis=1))
    logger.write(f"train_loss {train_loss} val_loss {val_loss} ")
    logger.write(f"val accuracy_score {accuracy} ")
    logger.write(" ")
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(),'best.pth')


+ ===== Epoch 1/50 ===== +



train_loss 1.3632821030086943 val_loss 1.3338165374902577 
val accuracy_score 0.33756345177664976 
 
+ ===== Epoch 2/50 ===== +



train_loss 1.2387222515212166 val_loss 1.2581836993877706 
val accuracy_score 0.5 
 
+ ===== Epoch 3/50 ===== +



train_loss 1.0439172983169558 val_loss 1.1604431867599487 
val accuracy_score 0.5406091370558376 
 
+ ===== Epoch 4/50 ===== +



train_loss 0.8189246210787033 val_loss 1.0637684005957384 
val accuracy_score 0.5964467005076142 
 
+ ===== Epoch 5/50 ===== +


...


train_loss 0.05624925991675506 val_loss 0.9899911562410684 
val accuracy_score 0.7969543147208121 
 
+ ===== Epoch 47/50 ===== +



train_loss 0.05410798910694817 val_loss 0.9908609696251985 
val accuracy_score 0.7969543147208121 
 
+ ===== Epoch 48/50 ===== +



train_loss 0.04592610070637116 val_loss 0.9890988227338171 
val accuracy_score 0.8020304568527918 
 
+ ===== Epoch 49/50 ===== +



train_loss 0.052408682141039116 val_loss 0.9830494107803902 
val accuracy_score 0.8020304568527918 
 
+ ===== Epoch 50/50 ===== +



train_loss 0.057560275758927075 val_loss 0.98839767555742 
val accuracy_score 0.8020304568527918 
 
